---
---
# **`CASE STUDY - 1` :: Healthcare Provider Fraudulent Detection**

<img src="Display_Pic.png" alt="drawing" width="950"/>

---
---


**This notebook contains the sentence embeddings generated on the ICD-9 Universal Diagnosis and Procedure Codes from the pre-trained Bio-BERT model performed on the publicly available dataset at [Kaggle](https://www.kaggle.com/datasets/rohitrox/healthcare-provider-fraud-detection-analysis) with the intent.**


**Kindly checkout below link for gaining BUSINESS related insights related to this problem ::**

- [Deck : Detailed Explanation](https://docs.google.com/presentation/d/1Thuw_eZskafkl9W3xYuEVsjgTJKFeEkzEfAmczH96Uw/)


**Kindly checkout below link for TECHNICAL design document ::**

- [Technical Document](https://docs.google.com/document/d/10z9xbn4dZWkforlAszCDaa1M0roAW-MOf2pjJHSnEXk/)

# **`Notebook Contents`**

>[CASE STUDY - 1 :: Healthcare Provider Fraudulent Detection](#scrollTo=naPWqtUVigTd)

>[Notebook Contents](#scrollTo=PHkkU2ExpgPR)

>>[Downloading_Train_Data_Files](#scrollTo=Ry-TJ1anmAdO)

>>[Importing_Libraries](#scrollTo=ThPD8j4PnuVH)

>>>[VECTOR_Embeddings](#scrollTo=xhJqQq7R5XSz)

>>>>[Pre-processing the CODES](#scrollTo=Er0qcZNk5XS8)

>>[SUMMARY](#scrollTo=tbWnhQ0qgiCm)



## **Downloading_Train_Data_Files**

In [ ]:
## TRAIN set files
!gdown 12zSQN2FOxmuXFhz2xzPNussPisEfVP5w
!gdown 13XyBakfHiG-BNQPrYFXAHlsOcfICOTpx
!gdown 1dLxl4vkykPcm4Zj0abYR0Ohr7STQHg-1
!gdown 1rFER-7VuYb7GfCYeJrfxPidgK0lwqw3R
!gdown 1092Kqiqo1VFhVOgy2ttuW4Vu1pRTEOKp
!gdown 1mmzARIlfF3TkZQljk74bZ6xkl7azyy3K

## TEST set files
!gdown 11TPcDjNF5Ptk1bf3c7qM5RLaSfZIbXV2
!gdown 1PvYHPMxQNNj-E1VvNhGF7nHOiCe9J-xm
!gdown 1RxMPDYXRoRQxUzw9ly1D68Oe2ONIiQbm
!gdown 1naCnYPra543q1omchA_Y0Xir5twojaeA

Downloading...
From: https://drive.google.com/uc?id=12zSQN2FOxmuXFhz2xzPNussPisEfVP5w
To: /content/Train_Outpatientdata-1542865627584.csv
100% 77.4M/77.4M [00:00<00:00, 113MB/s] 
Downloading...
From: https://drive.google.com/uc?id=13XyBakfHiG-BNQPrYFXAHlsOcfICOTpx
To: /content/Train_Beneficiarydata-1542865627584.csv
100% 11.4M/11.4M [00:00<00:00, 40.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dLxl4vkykPcm4Zj0abYR0Ohr7STQHg-1
To: /content/Train-1542865627584.csv
100% 87.1k/87.1k [00:00<00:00, 78.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rFER-7VuYb7GfCYeJrfxPidgK0lwqw3R
To: /content/Train_Inpatientdata-1542865627584.csv
100% 8.57M/8.57M [00:00<00:00, 62.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1092Kqiqo1VFhVOgy2ttuW4Vu1pRTEOKp
To: /content/CMS32_DESC_LONG_SHORT_SG.xlsx
100% 194k/194k [00:00<00:00, 65.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mmzARIlfF3TkZQljk74bZ6xkl7azyy3K
To: /content/CMS32_DESC_LONG_SHORT_DX.xlsx
10

## **Importing_Libraries**

In [ ]:
pip install pyspark==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 29 kB/s 
     |████████████████████████████████| 198 kB 65.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=b7ce1c4e9f5505babc0708bf5a4cdb4d77cf04b42193250c8a2958c402439fe6
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
pip install nlu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 570 kB 12.7 MB/s 
     |████████████████████████████████| 532 kB 77.1 MB/s 


In [ ]:
import nlu
import os
import sys
import math
import scipy as scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### **`VECTOR_Embeddings`**

- Generating the vector embeddings of ::
    - `CLAIM Admit Diagnosis Codes`
    - `Diagnosis Codes`
    - `Procedure Codes`


- For now, I'm not including the `Dx Related Group Code` for generating the similarity score b/w these features.

In [ ]:
# Importing the ICD-9 Dx and Proc Codes Descriptions
CAD_Diag_codes_df = pd.read_excel("CMS32_DESC_LONG_SHORT_DX.xlsx", dtype="str")
PROC_codes_df = pd.read_excel("CMS32_DESC_LONG_SHORT_SG.xlsx", dtype="str")

In [ ]:
# Will use the Long Descriptions
CAD_Diag_codes_df.drop(['SHORT DESCRIPTION'], axis=1, inplace=True)
PROC_codes_df.drop(['SHORT DESCRIPTION'], axis=1, inplace=True)

In [ ]:
# Renaming the columns
CAD_Diag_codes_df.columns = ['Dx_Code', 'Dx_Desc']
PROC_codes_df.columns = ['Proc_Code', 'Proc_Desc']

In [ ]:
# Few CAD and Dx codes and their descriptions
CAD_Diag_codes_df.head()

,Dx_Code,Dx_Desc
0,0010,Cholera due to vibrio cholerae
1,0011,Cholera due to vibrio cholerae el tor
2,0019,"Cholera, unspecified"
3,0020,Typhoid fever
4,0021,Paratyphoid fever A


In [ ]:
# Few PROC codes and their descriptions
PROC_codes_df.head()

,Proc_Code,Proc_Desc
0,0001,Therapeutic ultrasound of vessels of head and ...
1,0002,Therapeutic ultrasound of heart
2,0003,Therapeutic ultrasound of peripheral vascular ...
3,0009,Other therapeutic ultrasound
4,0010,Implantation of chemotherapeutic agent


In [ ]:
CAD_Diag_codes_df.shape, CAD_Diag_codes_df["Dx_Code"].nunique()

((14567, 2), 14567)

- There are 14,567 unique Diagnosis Codes.

In [ ]:
PROC_codes_df.shape, PROC_codes_df["Proc_Code"].nunique()

((3882, 2), 3882)

- There are 3882 unique Procedures Codes.

In [ ]:
print(CAD_Diag_codes_df.dtypes, "\n"*2, PROC_codes_df.dtypes)

Dx_Code    object
Dx_Desc    object
dtype: object 

 Proc_Code    object
Proc_Desc    object
dtype: object


#### **Pre-processing the CODES**

In [ ]:
# Printing some of the random CAD and Dx codes and their descriptions
random_cad_dx_desc = np.random.randint(low=0, high=CAD_Diag_codes_df.shape[0]-1, size=8)

for idx in random_cad_dx_desc:
    print("Dx_Code --> {} \nDx_Code Description --> {}".format(CAD_Diag_codes_df['Dx_Code'].iloc[idx], 
                                                                 CAD_Diag_codes_df['Dx_Desc'].iloc[idx]))
    print("===============================================================")

Dx_Code --> 6079 
Dx_Code Description --> Unspecified disorder of penis
Dx_Code --> 29699 
Dx_Code Description --> Other specified episodic mood disorder
Dx_Code --> 71652 
Dx_Code Description --> Unspecified polyarthropathy or polyarthritis, upper arm
Dx_Code --> 1542 
Dx_Code Description --> Malignant neoplasm of anal canal
Dx_Code --> 2409 
Dx_Code Description --> Goiter, unspecified
Dx_Code --> 5373 
Dx_Code Description --> Other obstruction of duodenum
Dx_Code --> 9895 
Dx_Code Description --> Toxic effect of venom
Dx_Code --> 37412 
Dx_Code Description --> Mechanical ectropion


In [ ]:
# Printing some of the random Proc codes and their descriptions
random_proc_codes_desc = np.random.randint(low=0, high=PROC_codes_df.shape[0]-1, size=8)

for idx in random_proc_codes_desc:
    print("Proc_Code --> {} \nProc_code Description --> {}".format(PROC_codes_df['Proc_Code'].iloc[idx], 
                                                                   PROC_codes_df['Proc_Desc'].iloc[idx]))
    print("===============================================================")

Proc_Code --> 8023 
Proc_code Description --> Arthroscopy, wrist
Proc_Code --> 5849 
Proc_code Description --> Other repair of urethra
Proc_Code --> 8341 
Proc_code Description --> Excision of tendon for graft
Proc_Code --> 5503 
Proc_code Description --> Percutaneous nephrostomy without fragmentation
Proc_Code --> 8734 
Proc_code Description --> Intrathoracic lymphangiogram
Proc_Code --> 3862 
Proc_code Description --> Other excision of vessels, other vessels of head and neck
Proc_Code --> 0972 
Proc_code Description --> Other repair of punctum
Proc_Code --> 8779 
Proc_code Description --> Other x-ray of the urinary system


In [ ]:
# Importing the libraries
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance

In [ ]:
# Downloading the required components
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Stopwords
eng_stopwords = stopwords.words('english')
eng_stopwords.remove('no')
eng_stopwords.remove('nor')
eng_stopwords.remove('not')

print(eng_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [ ]:
# Pre-processing the sentences
stp_words = set(eng_stopwords)
def preprocess_sentence(text, stop_words=stp_words):
  """
  Description : This function is created for pre-processing the sentences of ICD given codes description
  """
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()
    
    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

In [ ]:
# Pre-process every description sentence
CAD_Diag_codes_df['Dx_Desc_pp'] = CAD_Diag_codes_df['Dx_Desc'].apply(lambda row: preprocess_sentence(row))
PROC_codes_df['Proc_Desc_pp'] = PROC_codes_df['Proc_Desc'].apply(lambda row: preprocess_sentence(row))

In [ ]:
CAD_Diag_codes_df.head()

,Dx_Code,Dx_Desc,Dx_Desc_pp
0,0010,Cholera due to vibrio cholerae,cholera due vibrio cholerae
1,0011,Cholera due to vibrio cholerae el tor,cholera due vibrio cholerae el tor
2,0019,"Cholera, unspecified",cholera unspecified
3,0020,Typhoid fever,typhoid fever
4,0021,Paratyphoid fever A,paratyphoid fever


In [ ]:
PROC_codes_df.head()

,Proc_Code,Proc_Desc,Proc_Desc_pp
0,0001,Therapeutic ultrasound of vessels of head and ...,therapeutic ultrasound vessels head neck
1,0002,Therapeutic ultrasound of heart,therapeutic ultrasound heart
2,0003,Therapeutic ultrasound of peripheral vascular ...,therapeutic ultrasound peripheral vascular ves...
3,0009,Other therapeutic ultrasound,therapeutic ultrasound
4,0010,Implantation of chemotherapeutic agent,implantation chemotherapeutic agent


In [ ]:
# Loading the pre-trained Bio-BERT model
biobert_pt_model = nlu.load('en.embed_sentence.biobert.pubmed_pmc_base_cased')

sent_biobert_pubmed_pmc_base_cased download started this may take some time.
Approximate size to download 386.7 MB
[OK!]


In [ ]:
# Getting the sentence embeddings from the pre-trained Bio-BERT Model
CAD_Dx_codes_embeddings_df = biobert_pt_model.predict(CAD_Diag_codes_df['Dx_Desc_pp'], output_level='sentence')
Proc_codes_embeddings_df = biobert_pt_model.predict(PROC_codes_df['Proc_Desc_pp'], output_level='sentence')

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
CAD_Dx_codes_embeddings_df.shape, Proc_codes_embeddings_df.shape

((14567, 2), (3882, 2))

In [ ]:
CAD_Dx_codes_embeddings_df.head()

,sentence,sentence_embedding_biobert
0,cholera due vibrio cholerae,"[0.2308211326599121, 0.012130413204431534, -0...."
1,cholera due vibrio cholerae el tor,"[0.27864786982536316, 0.07002799957990646, -0...."
2,cholera unspecified,"[0.4682339131832123, 0.07824743539094925, -0.1..."
3,typhoid fever,"[0.18718130886554718, 0.1722823977470398, -0.1..."
4,paratyphoid fever,"[0.23444253206253052, 0.3054424822330475, -0.1..."


In [ ]:
Proc_codes_embeddings_df.head()

,sentence,sentence_embedding_biobert
0,therapeutic ultrasound vessels head neck,"[-0.07812043279409409, 0.12884767353534698, -0..."
1,therapeutic ultrasound heart,"[-0.013654773123562336, 0.13431645929813385, -..."
2,therapeutic ultrasound peripheral vascular ves...,"[0.008313322439789772, 0.14970511198043823, 0...."
3,therapeutic ultrasound,"[0.052392199635505676, 0.180985689163208, -0.0..."
4,implantation chemotherapeutic agent,"[-0.04786144942045212, 0.47679537534713745, -0..."


In [ ]:
CAD_Diag_codes_df['Dx_embeddings'] = CAD_Dx_codes_embeddings_df['sentence_embedding_biobert']
PROC_codes_df['Proc_embeddings'] = Proc_codes_embeddings_df['sentence_embedding_biobert']

In [ ]:
# Checking the length and type of embeddings vector
CAD_Diag_codes_df['Dx_embeddings'].iloc[0].shape, type(CAD_Diag_codes_df['Dx_embeddings'].iloc[0])

((768,), numpy.ndarray)

In [ ]:
# Preparing the dictionary of CAD and Dx CODE and its EMBEDDING
cad_diag_codes_embeddings_dict = dict(CAD_Diag_codes_df[['Dx_Code','Dx_embeddings']].values)

In [ ]:
# Checking the length of dict keys
len(cad_diag_codes_embeddings_dict.keys()), len(cad_diag_codes_embeddings_dict.values())

14567

In [ ]:
# Preparing the dictionary of PROC CODE and its EMBEDDING
proc_codes_embeddings_dict = dict(PROC_codes_df[['Proc_Code','Proc_embeddings']].values)

In [ ]:
# Checking the length of dict keys
len(proc_codes_embeddings_dict.keys()), len(proc_codes_embeddings_dict.values())

(3882, 3882)

In [ ]:
# Storing the CAD,Dx and PROC CODES DICTIONARIES physically
import pickle
with open('cad_diag_codes_embeddings.pkl', 'wb') as f:
    pickle.dump(cad_diag_codes_embeddings_dict, f)

with open('proc_codes_embeddings.pkl', 'wb') as f:
    pickle.dump(proc_codes_embeddings_dict, f)
      
with open('cad_diag_codes_embeddings.pkl', 'rb') as f:
    loaded_cad_dict = pickle.load(f)

with open('proc_codes_embeddings.pkl', 'rb') as f:
    loaded_proc_dict = pickle.load(f)

In [ ]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy it there
!cp cad_diag_codes_embeddings.pkl /content/drive/MyDrive/AAIC_Case_Study_1/Dataset
!cp proc_codes_embeddings.pkl /content/drive/MyDrive/AAIC_Case_Study_1/Dataset

## **`SUMMARY`**

- I also tried using SENT2VEC model pre-tarined on Biomedical PubMed and PMC dataset that internaly uses FastText as its framework.
  - But, the model was so huge that it was crashing my system RAM.


- I also tried using Bio-BERT pre-trained models trained on PubMed and PMC datasets available on HuggingFaces.
  - And, these were giving some configuration related errors. Also, I checked on internet that people have raised some out-of-memory concerns related to these as well.

- Thus, I ended up using the NLU library from John Snow Labs NLU that provides SOTA for NLP&NLU with 5000+ of pretrained models in 200+ languages. 
  - It enables swift and simple development and research with its powerful Pythonic and Keras inspired API. It is powerd by John Snow Labs powerful Spark NLP library.